# Lecture 3.2:  Data transformation

<div style="border: 1px double black; padding: 10px; margin: 10px">

**Goals for today's lecture:**
* Continue to learn [how to manipulate data](#Data-manipulation), including:
    * Pipes
    * Adding New Variables
    
This lecture note corresponds to Chapter 5.5 of your book.
</div>


Let us load up the `tidyverse` and `nycflights13` packages.

In [2]:
library(tidyverse)
library(nycflights13)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Pipes
Starting now, we will make extensive use of the pipe operator `%>%`. 

### How `%>%` works
Under the hood, `x %>% f(y)` turns into `f(x, y)`, and `x %>% f(y) %>% g(z)` turns into `g(f(x, y), z)` and so on. We can use `%>%` on any function, not just those defined in tidyverse.

Here is an example on printing "hello world" using pipe. 

In [3]:
"hello world" %>% print()  # prints "hello world"

[1] "hello world"


We will see the usefulness of pipe `%>%` later in the lecture as it greatly simplifies our code

# Adding New Variables
The `dplyr`/`tidyverse` package offers the `mutate()` and `transmute()` commands to add new variables to data tibbles. The syntax is:
```{r}
<tibble> %>% mutate(<new variable> = <formula for new variable>,
                    <other new variable> = <other formula>)
```
This returns a copy of `<tibble>` with the new variables added on `transmute()` does the same thing as `mutate()` but only keeps the new variables.

Let us zoom in on a few variables of interest.

In [4]:
my_flights <- select(flights, year:day, dep_time, arr_time, air_time, origin, dest)
head(my_flights)

year,month,day,dep_time,arr_time,air_time,origin,dest
<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<chr>
2013,1,1,517,830,227,EWR,IAH
2013,1,1,533,850,227,LGA,IAH
2013,1,1,542,923,160,JFK,MIA
2013,1,1,544,1004,183,JFK,BQN
2013,1,1,554,812,116,LGA,ATL
2013,1,1,554,740,150,EWR,ORD


Use Pipe `%>%` to create the table above. 

In [5]:
flights %>% select(year:day, dep_time, arr_time, air_time, origin, dest) %>% head()

year,month,day,dep_time,arr_time,air_time,origin,dest
<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<chr>
2013,1,1,517,830,227,EWR,IAH
2013,1,1,533,850,227,LGA,IAH
2013,1,1,542,923,160,JFK,MIA
2013,1,1,544,1004,183,JFK,BQN
2013,1,1,554,812,116,LGA,ATL
2013,1,1,554,740,150,EWR,ORD


The above code basically select the variables that we are interested in and save it into the object `my_flights`.  

Additional variable can be added using the `mutate()` function. We already have an `air_time` variable. Let us compute the total time for the flight by subtracting the time of departure `dep_time` from time of arrival `arr_time`.

In [6]:
mutate(my_flights, total_time = arr_time - dep_time) %>%
    head()

year,month,day,dep_time,arr_time,air_time,origin,dest,total_time
<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<int>
2013,1,1,517,830,227,EWR,IAH,313
2013,1,1,533,850,227,LGA,IAH,317
2013,1,1,542,923,160,JFK,MIA,381
2013,1,1,544,1004,183,JFK,BQN,460
2013,1,1,554,812,116,LGA,ATL,258
2013,1,1,554,740,150,EWR,ORD,186


Another way to do the same thing is by using pipe twice

In [7]:
flights %>% mutate(total_time = arr_time - dep_time) %>% 
            select(year, month, day, dep_time, arr_time, air_time, origin, dest, total_time) %>% head()

year,month,day,dep_time,arr_time,air_time,origin,dest,total_time
<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<int>
2013,1,1,517,830,227,EWR,IAH,313
2013,1,1,533,850,227,LGA,IAH,317
2013,1,1,542,923,160,JFK,MIA,381
2013,1,1,544,1004,183,JFK,BQN,460
2013,1,1,554,812,116,LGA,ATL,258
2013,1,1,554,740,150,EWR,ORD,186


We notice something odd though. When we subtract 5h 17m from 8h 30m we should get 3h 13m, i.e. 193 minutes. But instead we get 313 minutes below.

The issue is that `dep_time` and `arr_time` are in the hour-minute notation, so you cannot add and subtract them like regular numbers. We should first convert these times into the number of minutes elapsed since midnight.

We want add to new variables `new_dep` and `new_arr` but we need to write a function first that can do the conversion. The function is given below; we'll learn how it works later in the semester. For now just think of it as a black box that converts times from one format to another.

In [8]:
hourmin2min <- function(hourmin) {
    min <- hourmin %% 100 # quotient after division by 100
    hour <- (hourmin - min) %/% 100 # remainder after division by 100
    return(60*hour + min)
} 

Let us test the function on 530. That's 5h 30min, i.e., 330 minutes since midnight.

In [9]:
hourmin2min(530)

[1] 330

The `hourmin2min` function is **vectorized**: given a vector, it outputs a vector.

In [10]:
hourmin2min(c(430,530,630,730))

[1] 270 330 390 450

Let us now create two new variables obtained from `arr_time` and `dep_time` by converting them into minutes since midnight. In the same command, we can also create a new `total_time` column containing their difference.

In [11]:
my_flights_new <- mutate(my_flights, new_arr = hourmin2min(arr_time), new_dep = hourmin2min(dep_time))
head(my_flights_new)

year,month,day,dep_time,arr_time,air_time,origin,dest,new_arr,new_dep
<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
2013,1,1,517,830,227,EWR,IAH,510,317
2013,1,1,533,850,227,LGA,IAH,530,333
2013,1,1,542,923,160,JFK,MIA,563,342
2013,1,1,544,1004,183,JFK,BQN,604,344
2013,1,1,554,812,116,LGA,ATL,492,354
2013,1,1,554,740,150,EWR,ORD,460,354


Now we can subtract the departure time `new_dep` from the arrival time `new_arr` to get a new variable `total_time`.

In [12]:
my_flights_total <- mutate(my_flights_new, total_time = new_arr - new_dep)
head(my_flights_total)

year,month,day,dep_time,arr_time,air_time,origin,dest,new_arr,new_dep,total_time
<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
2013,1,1,517,830,227,EWR,IAH,510,317,193
2013,1,1,533,850,227,LGA,IAH,530,333,197
2013,1,1,542,923,160,JFK,MIA,563,342,221
2013,1,1,544,1004,183,JFK,BQN,604,344,260
2013,1,1,554,812,116,LGA,ATL,492,354,138
2013,1,1,554,740,150,EWR,ORD,460,354,106


Let us try to do the same thing using pipe just using one line of code.  In this code, we are only interested in the following variables -- `dep_time`, `arr_time`, `new_dep`, `new_arr`, and `total_time`.  

In [13]:
 mutate(flights, new_arr = hourmin2min(arr_time), new_dep = hourmin2min(dep_time),total_time = new_arr - new_dep
) %>% select(dep_time, arr_time, new_dep, new_arr, total_time) %>% head()

dep_time,arr_time,new_dep,new_arr,total_time
<int>,<int>,<dbl>,<dbl>,<dbl>
517,830,317,510,193
533,850,333,530,197
542,923,342,563,221
544,1004,344,604,260
554,812,354,492,138
554,740,354,460,106


## Up Next - Summarize Function and Case Study on Data Manipulation